In [40]:
import pandas as pd
age=63
fileArr=[
   './crimes/arson.csv', 
    './crimes/assault.csv',
    './crimes/fraud.csv',
    './crimes/gun-violence.csv',
    './crimes/harrassment.csv',
    './crimes/motor-vehicle-accidents.csv',
    './crimes/murder.csv',
    './crimes/robbery.csv',
    './crimes/sex-crime.csv',
    './crimes/drug-overdose-victims.csv',
    './crimes/Missing-Persons.csv',
    './crimes/Human-Trafficking.csv'
]
file=[]
for path in fileArr:
    file.append(pd.read_csv(path))
    # low=[]
    # up=[]
    # for index,row in df.iterrows():
    #     val=row[0].split('-')
    #     low.append(int(val[0]))
    #     up.append(int(val[1]))
    # df['low']=low
    # df['up']=up
    # df=df.iloc[:,1:]
    # df.to_csv(path,index=False)

In [41]:
crime_data=pd.read_csv('../wdc-precinct-wise-dataset.csv')
crime_data=crime_data.iloc[:,1:]
crimes=crime_data.columns
li=[]
# print(crime_data.head(2))
# for column in crime_data.columns:
    # print(crime_data.columns[column])
for df in file:
    for _,row in df.iterrows():
        key=row.keys()[0]
        if key in crimes:
            if age>=row['low'] and age<=row['up']:
                if key=='Robbery':
                    li.append({'Theft': row['Points']})
                    crime_data['Theft']=crime_data['Theft']*row['Points']
                li.append({key: row['Points']})
                crime_data[key]=crime_data[key]*row['Points']
                # print(crime_data.head(1))
                break
        else:
            break
if 'Others' in crimes:
    crime_data['Others']=crime_data['Others']*2.5
    li.append({'Others':2.5})
if 'Public Nuisance' in crimes:
    crime_data['Public Nuisance']=crime_data['Public Nuisance']*2.5
    li.append({'Public Nuisance':2.5})
print(li)
li=[]
crime_data=crime_data.fillna(0)
for index,row in crime_data.iterrows():
    li.append(sum([float(x) for x in row[2:].values]))
crime_data['total']=li
# crime_data
crime_data.head(2)

[{'Arson': 3.0}, {'Assault': 1.0}, {'Murder': 1.0}, {'Theft': 1.0}, {'Robbery': 1.0}, {'Sex Crime': 1.0}]


,precinct,total,Theft,Robbery,Murder,Assault,Sex Crime,Arson
0,401,9.0,8.0,1.0,0.0,0.0,0.0,0.0
1,201,5.0,4.0,1.0,0.0,0.0,0.0,0.0


In [42]:
import json
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry import shape
precincts=json.load(open('../first-phase/Washington DC/WashingtonDC-precinct.geojson','r'))
# precincts=precincts['features']
# print(len(precincts))

# polygons=[]
# # precincts[0]
# for precinct in precincts['features']:
#   precinct['precinct']=precinct['properties']['precinct']
#   polygons.append([shape(precinct['geometry']),precinct['properties']['precinct']])

In [43]:
import plotly.express as px
import plotly.io as pio
import json
pio.renderers.default='chrome'
# crime_data=pd.read_csv('seattle-precinct-wise-dataset.csv')
data=json.load(open('../washington-city.json'))
lat=data['latitude']
long=data['longitude']

In [44]:
fig=px.choropleth_mapbox(crime_data,locations='precinct',featureidkey="properties.precinct",geojson=precincts,color='total',color_continuous_scale='ylorrd')
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(mapbox_style="carto-positron", 
                  mapbox_zoom=9.5,
                  mapbox_center={"lat": lat, "lon": long},
                  margin={"r":0,"t":0,"l":0,"b":0},
                  uirevision='constant')
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_html('../first_figure2.html')